In [ ]:
import numpy as np
import Threshold_strength_responsibility_model_functions as TM
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import pickle
import networkx as nx
import random
import os

%load_ext autoreload
%autoreload 2

In [ ]:
dataset = 'pr_school'

with open('../../data/pairwise/' + dataset + '_strength_edgelist_pairwise_sorted_norm.pkl', 'rb') as f:
    s_edgelist = pickle.load(f)

In [ ]:
G = nx.Graph()
for edge in list(s_edgelist.keys()):
    G.add_weighted_edges_from([tuple((edge[0],edge[1],s_edgelist[edge]))])
nb_nodes = len(list(G.nodes))

In [ ]:
thresholds = [0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.22]
if dataset == 'pr_school':
    thresholds = [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16]

mu = 0.1

In [ ]:
times = 10000

### Simulate

In [ ]:
def save_on_csv(filename, variable_list, writing_operation):
    with open(filename, writing_operation) as csvfile:
        writer = csv.writer(csvfile)
        try:
            [writer.writerow(s) for s in variable_list]
        except:
            writer.writerow(variable_list)

In [ ]:
folder = 'results/inf_treeSIR/'
if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
tm = TM.Threeshold_strength_resp_model(s_edgelist)

for threshold in thresholds:
    nb_good_sim = 0
    print('threshold',threshold)
    C_list = np.full(len(s_edgelist)*2,0)
    t_i = [[] for i in range(nb_nodes)]

    for n in tqdm(range(times)):
        pat0 = random.randint(0,nb_nodes-1) # choose seed (patient 0)
        # initialize:
        tm.initialize_time0(pat0)
        # simulate:
        C_dict, flag_inf, t_i = tm.simulate(threshold,mu,t_i)
        C_list = C_list + np.array(list(C_dict.values()))
        if flag_inf:
            nb_good_sim += 1
            if nb_good_sim == 2000:
                break
    print('nb_good_sim',nb_good_sim)
        
             
    # save infection pattern:
    C_list = C_list/nb_nodes
    file_name = folder + "C_mean_weighted_%s_th_%.2f_mu_%.3f.csv"%(dataset,threshold,mu)
   
    save_on_csv(file_name,C_list,'w')